In [1]:
import platform
import random
import time
import sys
import pickle
import numpy as np

from pysem.corpora import SNLI
from pysem.networks import DependencyNetwork
from pysem.utils.ml import MultiLayerPerceptron
from pysem.utils.snli import CompositeModel
from itertools import islice

class PickleDummy(object):

    def __init__(self):
        pass

    def __call__(self, x):
        return x

snli = SNLI('/Users/peterblouw/corpora/snli_1.0/')
snli.load_vocab('snli_words.pickle')

dim = 300
pretrained = 'pretrained_snli_embeddings.pickle'



In [2]:
encoder = DependencyNetwork(dim=dim, vocab=snli.vocab, pretrained=pretrained)
classifier = MultiLayerPerceptron(di=2*dim, dh=dim, do=3)

model = CompositeModel(snli, encoder, classifier)


In [3]:
model.train(iters=1, bsize=1, rate=0.005, log_interval=2, schedule=12000)

for item in model.encoder.__dict__.keys():
    print(item, type(model.encoder.__dict__[item]))

biases <class 'collections.defaultdict'>
wgrads <class 'collections.defaultdict'>
vocab <class 'list'>
vectors <class 'dict'>
bgrads <class 'collections.defaultdict'>
dwm <class 'numpy.ndarray'>
tree <class 'list'>
wm <class 'numpy.ndarray'>
rate <class 'float'>
weights <class 'collections.defaultdict'>
xgrads <class 'collections.defaultdict'>
dim <class 'int'>


In [4]:
with open('classifier.pickle', 'wb') as pfile:
	pickle.dump(model.classifier, pfile)

model.encoder.save('encoder.pickle')


test_encoder = DependencyNetwork(dim=20, vocab=['test'])
test_encoder.load('encoder.pickle')

# with open('data.pickle', 'wb') as pfile:
# 	test_data = pickle.load(pfile)
    
with open('classifier.pickle', 'rb') as pfile:
	test_classifier = pickle.load(pfile)
    
test_model = CompositeModel(snli, test_encoder, test_classifier)

    
print('Testing Model Loading')
print(test_model.dnn_accuracy(test_model.test_data))

Testing Model Loading
0.3291938110749186


In [5]:


print(model.dnn_accuracy(model.test_data))


    


0.3291938110749186
